Could look like:

1. For each order 1 peak, select the TES for which it can be measured
2. Measure by hand its position for each TES for each order 1 peak
3. Define a frequency resolution and create the theoretical synthbeam profile for each frequency
4. At each position related to the frequency measured, get the flux in a zone on the order 1 peak and around at the same elevation (the old elevation, not the delta_elevation)
5. At each elevation, compute the atmosphere temperature and deduce the transmission
6. Fit all frequencies together with weights on the synth beam that are the Moon emission
7. Stack the results for all TES and all order 1 peaks

Questions/remarks:

- the pixel size determines the frequency dependency (?)
- for each pixel we get the corresponding frequency and we compare the signal with the QUBIC instrument beam at that frequency times the Moon emission (what about neighbour frequencies?)
- for each order 1 peak, I create a list with all TES that have it
- I note the order 1 peak position manually for all these
- the fit is applied on each pixel seperately and each order 1 peak seperately and I take the weighted mean or median of all TES? weights to be determined with std of flat space between orders 0 and 1
- to get a better understanding of atmosphere, measure flux at two other positions with same elevation?
- sort out the thing in Giuseppe's paper with the emission/transmission of atm: is it that we can understand the transmission of atm from the emission measured?
- how to know what is atmosphere temperature (atmosphere emission) and simply TES bath temperature changing? --> need to find the TES bath temperature data to remove this trend?
- the order 0 and order 1 peaks don't seem to have the same amplitude ratio as the theoretical synthbeam
- use the filter characteristics from QUBIC III

## Imports and notebook configuration

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%config InlineBackend.figure_format='retina'
from IPython import display
display.display(display.HTML("<style>.container { width:95% !important; }</style>"))

# %matplotlib inline
%matplotlib ipympl
# %matplotlib widget

### General imports
import os
import sys
import time
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patch
import healpy as hp
import pickle

plt.rc('figure',figsize=(10, 6))
plt.rc('font',size=12)

### Astropy configuration
from astropy.visualization import quantity_support
quantity_support()
import astropy.units as u



#### QUBIC IMPORT
from qubic.lib import Qdictionary 
from qubic.lib.Instrument import Qinstrument




def iQS2iQP(indexQS):
    qpnumi, qpasici = qp.pix2tes.pix2tes(indexQS+1)
    return qpnumi+(qpasici-1)*128-1

def iQP2iQS(indexQP):
    QStesnum = qp.pix2tes.tes2pix(indexQP%128+1, indexQP//128+1)
    return QStesnum-1

d = Qdictionary.qubicDict()
dictfilename = '/dicts/global_source_oneDet.dict'
d.read_from_file(dictfilename)
q = Qinstrument.QubicInstrument(d)

# plt.rc('figure',figsize=(20,20))
plt.rc('font',size=12)


### Temporary update of the path 
in order to be able to load libraries that are still in development and not yet in the QUBIC path. This will have to be removed when the relevant libraries are finalized and integrated into QubicSoft

In [ ]:
dirtemplibs = ["/Users/huchet/qubic/qubic/scripts/MoonProject/", "/Users/huchet/Documents/code/scripts/", "/Users/huchet/Documents/code/data/"] #[os.environ['QUBIC_DATADIR']+'scripts/MoonProject/']
for rep in dirtemplibs:     
    if rep not in sys.path:
        sys.path.append(rep)

#### Local files that will need to be installed in the Qubic Libs
# import fitting as fit
# import time_domain_tools as tdt
# import useful_functions as uf
import qubic.scripts.MoonProject.pipeline_moon_plotting as pmp
import qubic.scripts.MoonProject.pipeline_moon_functions as pmf

In [ ]:
mydatadir = '/Users/huchet/Documents/code/data/ComissioningTD/'
mydatadir2 = "/Users/huchet/Documents/code/scripts/MoonProject/"
mydatadir3 = "/Users/huchet/Documents/code/data/"

### Observation date and corresponding file

In [ ]:
ObsDate = '2022-07-14'
ObsSession = 0
dirs = glob.glob(mydatadir + ObsDate + '/*')
print(dirs)
datadir = dirs[0]

### Observing Site

## Get good maps and Moon positions on TES in Moon coordinates

In [ ]:
raw = False
RADec = False
nside = 256
azel = "v1"

if raw: raw_str = "_raw"
else: raw_str = ""
if RADec: RADec_str = "_RADec"
else: RADec_str = ""
if nside == 128: nside_str="_nside128"
else: nside_str = ""
if azel == "v1": azel_str = ""
else: azel_str = "_azel{}".format(azel)

filename = "202506-allmaps_moonpos-Jan14-2022_unturned{}{}{}{}.pkl".format(RADec_str, raw_str, nside_str, azel_str)
allTESNum, allmaps, _, visibly_ok_arr = pickle.load( open( mydatadir2 + filename, "rb" ) )
# filename_maps = "202506-allmaps-Jan14-2022_unturned{}{}.pkl".format(RADec_str, raw_str) # just the maps recomputed to try to get ht visibly not ok maps back
# allTESNum, allmaps = pickle.load( open( mydatadir2 + filename_maps, "rb" ) )
filename_fit = "202506-allmaps_moonpos-Jan14-2022_unturned{}{}.pkl".format(RADec_str, azel_str)
_, _, moon_fit, _ = pickle.load( open( mydatadir2 + filename_fit, "rb" ) )

# allTESNum, allmaps, moon_fit, visibly_ok_arr = pickle.load( open( mydatadir2 + "202506-allmaps_moonpos-Jan14-2022_unturned.pkl", "rb" ) )
# allTESNum, allmaps, _, visibly_ok_arr = pickle.load( open( mydatadir2 + "202506-allmaps_moonpos-Jan14-2022_unturned_raw.pkl", "rb" ) )
# allTESNum, allmaps, moon_fit, visibly_ok_arr = pickle.load( open( mydatadir2 + "202506-allmaps_moonpos-Jan14-2022_unturned_RADec.pkl", "rb" ) )

In [ ]:
%%script echo skipping
plt.figure()
n0 = 8
for i in range(len(allTESNum)):
    ok = allmaps[i,:] != hp.UNSEEN
    if (i%(n0**2))==0 and i !=0: 
        plt.tight_layout()
        plt.show()
        plt.figure()
    mm = np.zeros(12*nside**2)+ hp.UNSEEN
    mm[ok] = allmaps[i,ok]#-tdt.get_mode(allmaps[i,ok])
    mm[~ok] = hp.UNSEEN
    hp.gnomview(mm, reso=10, sub=(n0,n0,(i%(n0**2))+1), min=-5e3, max=2.5e4, 
                title=allTESNum[i], rot=[0, 0])
plt.show()

In [ ]:
# allTESNum, allmaps_2, moon_fit, visibly_ok_arr = pickle.load( open( mydatadir2 + "202506-allmaps_moonpos-Jan14-2022_unturned_raw.pkl", "rb" ) )

In [ ]:
# _, _, moon_fit, _ = pickle.load( open( mydatadir2 + "202506-allmaps_moonpos-Jan14-2022_unturned.pkl", "rb" ) )

In [ ]:
# np.all(allmaps == allmaps_2)

### Observing Site

In [ ]:
Salta_CNEA = {'lat':-24.731358*u.deg,
              'lon':-65.409535*u.deg,
              'height':1152*u.m,
              'UTC_Offset':-3*u.hour}
Obs_Site = Salta_CNEA

In [ ]:
az_qubic = 116.4
start_tt = 10000
speedmin = 0.1

try:
    data
except:
    data=None
    
data, tt, tinit, alltod, azt, elt, newazt, newelt, scantype, Tbath = pmf.format_data(az_qubic, start_tt, Obs_Site, speedmin, data, datadir)
# tinit = 1657842857.0064104
azmoon, elmoon = pmf.get_azel_moon(Obs_Site, tt, tinit, doplot=False)

In [ ]:
print(moon_fit[:][0])

In [ ]:
moon_azt = np.array([moon_fit[i][0][0] for i in range(len(moon_fit))])
moon_elt = np.array([moon_fit[i][0][1] for i in range(len(moon_fit))])
no_moon_azt = moon_azt - 3 # 3 degrees from the Moon, same elevation
moon_amp_arr = np.array([moon_fit[i][2] for i in range(len(moon_fit))])

### Line of sight elevation

In [ ]:
# need to add the gaussian illumination in all the following plots!

real_elt_arg = [np.argwhere(np.isclose(newelt, np.full_like(newelt, moon_elt[i]))) for i in range(len(moon_elt))]
real_los_elt = np.array([np.mean(elt[real_elt_arg[i]]) for i in range(len(moon_elt))])
plt.figure()
plt.scatter(real_los_elt[visibly_ok_arr], moon_amp_arr[visibly_ok_arr])
# plt.yscale("log")
plt.xlabel("Line of sight elevation")
plt.ylabel("Moon amplitude")
plt.show()

### Moon elevation

In [ ]:
real_moon_elt = np.array([np.mean(elmoon[real_elt_arg[i]]) for i in range(len(moon_elt))])
plt.figure()
plt.scatter(real_moon_elt[visibly_ok_arr], moon_amp_arr[visibly_ok_arr])
# plt.yscale("log")
plt.xlabel("Moon elevation")
plt.ylabel("Moon amplitude")
plt.show()

### TES bath temperature

In [ ]:
bath_temp_time = np.array([np.mean(Tbath[real_elt_arg[i]]) for i in range(len(moon_elt))])
plt.figure()
plt.scatter(bath_temp_time[visibly_ok_arr], moon_amp_arr[visibly_ok_arr])
# plt.yscale("log")
plt.xlabel("TES bath temperature")
plt.ylabel("Moon amplitude")
plt.show()

In [ ]:
print(moon_azt, moon_elt)

In [ ]:
%%script echo skipping

# compare the signal in a circle on the Moon and a circle next to it

sample_radius = 0.25 # degree

S_moon = np.zeros(len(moon_fit))
S_atm = np.zeros_like(S_moon)
for i in range(len(moon_fit)):
    if i != 151: #not visibly_ok_arr[i]:
        continue
    print("TES {}".format(i + 1))
    _, _ , tod_i = pmf.make_coadded_maps_TES(tt, alltod[i], azt, elt, scantype, newazt, newelt, nside=nside, doplot=False, check_back_forth=False, also_tod=True)
    mask_elt = (elt >= moon_elt[i] - sample_radius) & (elt <= moon_elt[i] + sample_radius)
    mask_moon = mask_elt & (azt >= moon_azt[i] - sample_radius) & (elt <= moon_azt[i] + sample_radius)
    mask_no_moon = mask_elt & (azt >= no_moon_azt[i] - sample_radius) & (elt <= no_moon_azt[i] + sample_radius)
    S_moon[i] = np.mean(tod_i[mask_moon])
    S_atm[i] = np.mean(tod_i[mask_no_moon])

In [ ]:
NumTES = 152 # 73, 152, 64 (ordre pics différent), 209 and 210 worked for Giuseppe!, 67 (3 peaks)
iTES = NumTES - 1

In [ ]:
# hp.gnomview(allmaps[iTES], rot=[0,0], reso=8, title='TES {}'.format(NumTES), min = -1e4, max=1e4)
# hp.gnomview(allmaps[iTES], rot=moon_fit[iTES][0], reso=10, title='TES {}'.format(NumTES), min = -1e4, max=1e4)
hp.gnomview(allmaps[iTES], rot=(0, 0), reso=10, title='TES {}'.format(NumTES), min = -1e4, max=1e4)
hp.graticule()
hp.projscatter(0, 0, marker='x', c="white")
hp.projplot([np.linspace(-2, 2), np.linspace(-3, 3)], marker='x', c="white")
plt.tight_layout()
plt.show()
print(moon_fit[iTES])

In [ ]:
# distance between two horns
delta_h = 14e-3 # m

# index in arrays for sb
order_0 = 24
order_1 = np.array([23, 30, 31, 32, 25, 18, 17, 16])

In [ ]:
order_0_pos = moon_fit[iTES][0]
ind_0 = order_0
# ind_1 = order_1[0]
if NumTES == 73:
    # order_1_pos_start = np.array([2.78, -1.74])
    order_1_pos_end = np.array([4.77, +0.02])
elif NumTES == 152:
    if azel == "v1":
        order_1_pos_end = np.array([-7.60, +6.63])
    if azel == "v2":
        order_1_pos_end = np.array([-7.14, +6.49])
    ind_1 = order_1[6] #17
elif NumTES == 99:
    order_1_pos_end = np.array([3.11, -3.25])
elif NumTES == 64:
    order_1_pos_end = np.array([4.57, -0.31])
elif NumTES == 209:
    # order_1_pos_end = np.array([-4.17, -3.49])
    order_1_pos_end = np.array([-4.04, -3.37])
    # order_1_pos_end = np.array([-3.95, -3.56])
    if RADec:
        order_1_pos_end = np.array([2.43, 4.80])
        order_0_pos = np.array([-2.95, -2.35])
    ind_1 = order_1[0]

elif NumTES == 210:
    order_1_pos_end = np.array([-4.45, -3.52])
    # order_1_pos_end = np.array([-4.04, -3.37])

print(order_0_pos, order_1_pos_end)        
print(order_1_pos_end - order_0_pos)
delta_angle_deg = 1
npoints = 1000
dist_order_0_1_deg = 9.5 # degrees

In [ ]:
all_az = np.array([pos[0] for pos in [order_0_pos, order_1_pos_end]])
all_el = np.array([pos[1] for pos in [order_0_pos, order_1_pos_end]])
order_1_pos_end

In [ ]:
# el = a * az + b
a = (np.mean(all_el[1:]) - all_el[0])/(np.mean(all_az[1:]) - all_az[0])
b = all_el[0] - a * all_az[0]

In [ ]:
delta_az = 1
x = np.linspace(moon_fit[iTES][0][0] + delta_az, order_1_pos_end[0] - delta_az, npoints) # azimuth
y = a*x + b

# Extract the values along the line, using cubic interpolation
zi = hp.get_interp_val(allmaps[iTES], x, y, lonlat=True)

extra_shift = 0#np.radians(0.05)

order_0_pos_sph = np.array([np.radians(90 - order_0_pos[1]), np.radians(order_0_pos[0])]) - extra_shift
order_1_pos_end_sph = np.array([np.radians(90 - order_1_pos_end[1]), np.radians(order_1_pos_end[0])]) - extra_shift

print(order_0_pos)
print(np.degrees(order_0_pos_sph))

# why does it fit better theory with ang_dist_deg=dist_order_0_1_deg-0.5 degree?
zi_test, theta_gc_test, phi_gc_test, angle_array_test = pmf.get_values_line(allmaps[iTES], order_0_pos_sph, order_1_pos_end_sph, npoints=npoints, ang_dist_deg=dist_order_0_1_deg, delta_angle_deg=delta_angle_deg)
# _, theta_gc_test, phi_gc_test = get_values_line(allmaps[iTES], order_0_pos_sph, order_1_pos_end_sph, npoints=npoints, delta_angle_deg=delta_angle_deg)
# zi_test, zi_test_full, theta_gc_test_full, phi_gc_test_full = get_values_large_line(allmaps[iTES], theta_gc_test, phi_gc_test, width_angle_deg=0.1, nlines=3)

zi_test[zi_test**2>1e20] = np.NaN

nlines = 4

if RADec:
    side = "theta"
else:
    side = "phi"

zi_test2, zi_test3, coord_2, coord_3 = pmf.get_side_lines(allmaps[iTES], theta_gc_test, phi_gc_test, side=side, nlines=nlines)


#-- Plot...
hp.gnomview(allmaps[iTES], reso=8, rot=(0, 0, 0), return_projected_map=True, no_plot=False).data
hp.projscatter(x, y, marker='.', c="r", lonlat=True)
hp.projscatter(theta_gc_test, phi_gc_test, marker='.', c="g", lonlat=False)

iline = 0
hp.projscatter(coord_2[0][iline], coord_2[1][iline], marker='.', c="b", lonlat=False)
hp.projscatter(coord_3[0][iline], coord_3[1][iline], marker='.', c="b", lonlat=False)

# if not RADec:
#     hp.projscatter(theta_gc_test, phi_gc_test_diff_up[0], marker='.', c="b", lonlat=False)
#     hp.projscatter(theta_gc_test, phi_gc_test_diff_down[0], marker='.', c="b", lonlat=False)
# else:
#     hp.projscatter(theta_gc_test_diff_up[0], phi_gc_test, marker='.', c="b", lonlat=False)
#     hp.projscatter(theta_gc_test_diff_down[0], phi_gc_test, marker='.', c="b", lonlat=False)
# hp.projscatter(theta_gc_test_full[0], phi_gc_test_full[0], marker='.', c="b", lonlat=False)
plt.show()

In [ ]:
zi_test2_mean = np.median(zi_test2, axis=0)
zi_test3_mean = np.median(zi_test3, axis=0)
concatenated_zi = np.concatenate([zi_test2, zi_test3])
concatenated_zi[concatenated_zi**2 > 1e20] = np.NaN
zi_test_diff = zi_test - np.nanmean(concatenated_zi, axis=0)
fig, ax = plt.subplots()
# ax.plot(zi)
ax.plot(zi_test, label="zi_test")
# ax.plot(zi_test2_mean, label="zi_test2")
# ax.plot(zi_test3_mean, label="zi_test3")
ax.plot(zi_test_diff, label="zi_test_diff")
plt.legend()
plt.show()

In [ ]:
%%script echo skipping
plt.figure()
plt.plot(all_az, all_el)
plt.scatter(all_az, all_el)
plt.plot(all_az, a *all_az + b)
plt.show()

In [ ]:
from qubic.lib.Qdictionary import qubicDict
from qubic.lib.Instrument.Qinstrument import QubicInstrument
from qubic.lib.Qscene import QubicScene
import scipy.constants as cst

In [ ]:
def get_dict():
    dictfilename = 'qubic/qubic/dicts/global_source_oneDet.dict'
    d = qubicDict()
    d.read_from_file(dictfilename)
    d['config'] = 'TD'
    d['filter_nu']= 150000000000.0
    d['beam_shape'] = 'gaussian'  # can be 'gaussian', 'fitted_beam' or 'multi_freq'  
    d['synthbeam'] = None         # we put nothing
    # d['nside'] = 512              # To have nice SB maps
    d['nside'] = nside
    d['use_synthbeam_fits_file'] = False
    d['synthbeam_fraction'] = 1
    d['synthbeam_kmax'] = 3
    d["filter_relative_bandwidth"] = 0.30 # extended the filter relative bandwidth in order to account for the lab characterisation
    return d

In [ ]:
d = get_dict()

In [ ]:
print(d["filter_relative_bandwidth"])

In [ ]:
print(150 * (1 - 0.3/2), 150 * (1 + 0.3/2))

In [ ]:
def find_pos(size_pix, npix, pos, centre): # translate a position in physical units to a position in degrees
    return ((pos - (centre - npix * size_pix / 2 ))//size_pix).astype(int)

In [ ]:
def synth_beam(B_sky, B_det, pos_r, pos_n, lbda, P, f, delta_h, size_pix, npix_integration): # only one pos_n or only one pos_r
    x, y = np.radians(pos_r[..., 0]), np.radians(pos_r[..., 1])
    nx, ny = np.radians(pos_n[..., 0]), np.radians(pos_n[..., 1])
    npix = np.array(np.shape(B_sky))
    centre = np.array([0, 0])
    pix_n = find_pos(size_pix, npix, pos_n, centre)
    pix_r = find_pos(size_pix, npix, pos_r, centre)

    # print("pix_n", pix_n)
    # print(B_det[pix_n[..., 0], pix_n[..., 1]])

    res_before_int = B_sky[pix_r[..., 0], pix_r[..., 1]] * B_det[pix_n[..., 0], pix_n[..., 1]] * np.sin(P*np.pi*delta_h/lbda*(x/f - nx))**2 * np.sin(P*np.pi*delta_h/lbda*(y/f - ny))**2 / (np.sin(np.pi*delta_h/lbda*(x/f - nx))**2 * np.sin(np.pi*delta_h/lbda*(y/f - ny))**2)
    intermediate_sum = np.cumsum(res_before_int, axis=1) # sum over columns
    intermediate_sum = intermediate_sum[:, ::npix_integration] # get one out of npix_integration
    final_npix = npix//npix_integration
    intermediate_sum[:, 1 : final_npix[1]] = intermediate_sum[:, 1 : final_npix[1]] - intermediate_sum[:, 0 : final_npix[1] - 1] # this is the sum over npix_integration columns
    intermediate_sum = np.cumsum(intermediate_sum, axis=0)
    intermediate_sum = intermediate_sum[::npix_integration] # get one out of npix_integration
    intermediate_sum[1 : final_npix[0]] = intermediate_sum[1 : final_npix[0]] - intermediate_sum[0 : final_npix[0] - 1] # this is the sum over npix_integration columns
    integrated_res = intermediate_sum / npix_integration**2
    return integrated_res

In [ ]:
npix_x = 501
npix_y = 501

size_img = 30 # degrees
size_pix = size_img/npix_x

print("Size image is {} degrees".format(size_img))

In [ ]:
npix_integration = 10 # more pixels to then integrate
npix_x_int = npix_x * npix_integration
npix_y_int = npix_y * npix_integration
size_pix_int = size_pix / npix_integration

In [ ]:
reso = [12.9 / size_pix_int] # fwhm in pixels
xc, yc = npix_x_int//2, npix_y_int//2 # centre of the map in pixel
B_sky = pmf.gauss2D(npix_x_int, npix_y_int, xc, yc, reso, amp=None, normal=True)
B_det = pmf.gauss2D(npix_x_int, npix_y_int, xc, yc, reso, amp=None, normal=True)

In [ ]:
%%script echo skipping

plt.figure()
plt.imshow(B_sky)
plt.colorbar()
plt.show()

plt.figure()
plt.plot(B_sky[npix_x_int//2])
plt.show()


In [ ]:
print(size_pix)
print(size_pix_int)

In [ ]:
%%script echo skipping

mean_nu = 150e9
bw = 0.15*mean_nu
nus = np.linspace(mean_nu - bw, mean_nu + bw, 10)
lmbdas = cst.c/nus

# There might be a problem between x and y
# xx, yy = np.meshgrid(np.linspace(-npix_x//2, npix_x//2, npix_x_int)*size_pix, np.linspace(-npix_y//2, npix_y//2, npix_y_int)*size_pix)
# xx, yy = 30, 0.1
# print(q_instrument.horn.center[iTES])
det_pos = q_instrument.horn.center[iTES]
xx, yy = det_pos[0]/size_pix, det_pos[1]/size_pix
# nx, ny = 30, 0.1 # why does the synthbeam change so much with nx
nx, ny = np.meshgrid(np.linspace(-npix_x//2, npix_x//2, npix_x_int)*size_pix, np.linspace(-npix_y//2, npix_y//2, npix_y_int)*size_pix)

pos_n = np.array([nx, ny])
pos_r = np.array([xx, yy])
# pos_r = np.swapaxes(pos_r, 0, 2)
pos_n = np.swapaxes(pos_n, 0, 2)

sb = []
sb_tot = np.zeros((npix_x, npix_y))
for i_lmbda, lmbda in enumerate(lmbdas):
    sb.append(synth_beam(B_sky, B_det, pos_r, pos_n, lmbda, P=len(q_instrument.horn.center), f=q_instrument.optics.focal_length, delta_h=delta_h, size_pix=size_pix, npix_integration=npix_integration))
    sb_tot += sb[i_lmbda]

In [ ]:
%%script echo skipping

print(np.shape(sb))
print(np.min(sb), np.max(sb), np.mean(sb), np.std(sb))

In [ ]:
%%script echo skipping

plt.figure()
max_sb = np.max(sb_tot)
plt.imshow(sb_tot/max_sb, norm="log", vmin=1e-2, vmax=1e0)
# plt.imshow(sb_tot)#, vmin=-max_sb, vmax=max_sb)
plt.colorbar()
plt.show()

plt.figure()
plt.imshow(sb[0])
plt.show()

In [ ]:
def get_full_synthbeam(n_nus, d):
    # central_freq = 150.
    central_freq = d["filter_nu"]
    numin = central_freq * (1 - d['filter_relative_bandwidth']/2.)
    numax = central_freq * (1 + d['filter_relative_bandwidth']/2.)
    # n_nus = 10
    # nus = np.linspace(numin, numax, n_nus)
    nu_edges = np.geomspace(numin, numax, n_nus + 1)
    nus = (nu_edges[1:] + nu_edges[:-1])/2
    rel_bw = (nu_edges[1:] - nu_edges[:-1])/nus
    print(nus)
    print(rel_bw)

    thetas = np.zeros((n_nus, 248, (2*d['synthbeam_kmax']+1)**2))
    phis = np.zeros((n_nus, 248, (2*d['synthbeam_kmax']+1)**2))
    vals = np.zeros((n_nus, 248, (2*d['synthbeam_kmax']+1)**2))
    my_sbs = np.zeros((n_nus, 12*d['nside']**2))


    for i_nu in range(n_nus):
        sub_d = get_dict()
        print(nus[i_nu])
        sub_d['filter_nu'] = nus[i_nu]# * 1e9
        sub_d['filter_relative_bandwidth'] = rel_bw[i_nu]
        print("Uncorrected bandwidth = [{}, {}]".format(nus[i_nu] * (1 - d['filter_relative_bandwidth']/2.), nus[i_nu] * (1 + d['filter_relative_bandwidth']/2.)))
        print("Bandwidth = [{}, {}]".format(nus[i_nu] * (1 - sub_d['filter_relative_bandwidth']/2.), nus[i_nu] * (1 + sub_d['filter_relative_bandwidth']/2.)))
        q_instrument_i = QubicInstrument(sub_d, FRBW=d['filter_relative_bandwidth'])
        q_scene_i = QubicScene(sub_d)
        # q_instrument_i = QubicInstrument(d)
        # q_scene_i = QubicScene(d)
        print(sub_d["synthbeam_kmax"])
        # Here we get the unsorted positions of the peaks, that is the position of a given peak doesn't depend on its amplitude
        thetas[i_nu, :, :], phis[i_nu, :, :], vals[i_nu, :, :] = q_instrument_i._peak_angles_unsorted(q_scene_i, 
                                                    sub_d['filter_nu'], 
                                                    q_instrument_i.detector.center, 
                                                    q_instrument_i.synthbeam, 
                                                    q_instrument_i.horn, 
                                                    q_instrument_i.primary_beam)
        
        my_sbs[i_nu, :] = q_instrument_i.get_synthbeam(q_scene_i, iTES)
    
    return my_sbs, thetas, phis, vals, nu_edges, nus, rel_bw

In [ ]:
n_nus = 50
my_sbs, thetas, phis, vals, nu_edges, nus, rel_bw = get_full_synthbeam(n_nus, d=d)

In [ ]:
inu = 0
# point_A = spherical2cartesian(1, lon[inu, iTES, 0], lat[inu, iTES, 0])
# point_B = spherical2cartesian(1, lon[inu, iTES, 1], lat[inu, iTES, 1])
point_A = pmf.spherical2cartesian(1, thetas[inu, iTES, ind_0], phis[inu, iTES, ind_0])
point_B = pmf.spherical2cartesian(1, thetas[inu, iTES, ind_1], phis[inu, iTES, ind_1])
sphere_centre=np.array([0, 0, 0])

print(point_A)
print(point_B)

great_circle_traj, _ = pmf.get_great_circle_traj(point_A, point_B, sphere_centre=sphere_centre, sphere_radius=1, npoints=npoints, ang_dist_deg=dist_order_0_1_deg, delta_angle_deg=delta_angle_deg)
theta_gc = np.arccos(great_circle_traj[:, 2])
phi_gc = np.arctan2(great_circle_traj[:, 1], great_circle_traj[:, 0])

print(np.degrees(theta_gc[0]))
print(np.degrees(phi_gc[2]))

In [ ]:
lon, lat = hp.pixelfunc.thetaphi2lonlat(thetas, phis)
# print(lon[inu, iTES])
# print(lat[inu, iTES])

two_peaks_az = lon[inu, iTES, [ind_0, ind_1]] + 180
two_peaks_el = lat[inu, iTES, [ind_0, ind_1]] + 180

# el = a * az + b
a_2 = (two_peaks_el[1] - two_peaks_el[0])/(two_peaks_az[1] - two_peaks_az[0])
b_2 = two_peaks_el[0] - a_2 * two_peaks_az[0]

delta_az = 2
x_2 = np.linspace(two_peaks_az[0] - delta_az, two_peaks_az[1] + delta_az, npoints) # azimuth
y_2 = a_2*x_2 + b_2

x_2 -= 180
y_2 -= 180

hp.gnomview(np.log10(my_sbs[inu]/np.max(my_sbs[inu])), rot=[0,90], reso=20, min=-5, max=0,
             title='Theory {} {} GHz: TES #{}'.format(d['config'], d['filter_nu']/1e9,iTES))
variation = vals[inu, iTES, :]/np.max(vals[inu, iTES, :])
hp.projscatter(thetas[inu, iTES, :], phis[inu, iTES, :], c=variation, alpha=variation**(1/4), marker='x', cmap='Reds')
hp.projscatter(x_2, y_2, marker='x', c="r", lonlat=True)
hp.projscatter(theta_gc, phi_gc, marker='x', c="g")
# for i_peak in range(np.shape(thetas)[2]):
#     hp.projtext(thetas[inu, iTES, i_peak], phis[inu, iTES, i_peak], 
#                 '{0:5.0f}: {1:4.2f}'.format(i_peak, vals[inu, iTES,i_peak]/np.max(vals[inu, iTES,:])), c='w', fontsize=8)
plt.show()

In [ ]:
%%script echo skipping

# Extract the values along the line, using cubic interpolation
# zi_2 = hp.get_interp_val(my_sbs[inu], x_2, y_2, lonlat=True)
zi_2 = hp.get_interp_val(my_sbs[inu], theta_gc, phi_gc)
zi_0 = hp.get_interp_val(my_sbs[-1], theta_gc, phi_gc)
print(np.shape(zi_2))

# fig, ax = plt.subplots()
# ax.plot(x_2, y_2)
# plt.show()

fig, ax = plt.subplots()
ax.plot(zi_0)
ax.plot(zi_2)
plt.show()

In [ ]:
sb = np.sum(my_sbs, axis=0)
hp.gnomview(np.log10(sb/np.max(sb)), rot=[0,90], reso=20, min=-5, max=0,
             title='Theory {} {} GHz: TES #{}'.format(d['config'], d['filter_nu']/1e9, iTES + 1))
for i_nu in range(n_nus):
    variation = vals[i_nu, iTES,:]/np.max(vals[i_nu, iTES,:])
    hp.projscatter(thetas[i_nu, iTES,:], phis[i_nu, iTES,:], c=variation, alpha=variation**(1/4), marker='x', cmap='Reds')
    for i_peak in range(np.shape(thetas)[2]):
        hp.projtext(thetas[inu, iTES, i_peak], phis[inu, iTES, i_peak], 
                    '{0:5.0f}: {1:4.2f}'.format(i_peak, vals[i_nu, iTES,i_peak]/np.max(vals[:, iTES,:])), c='w', fontsize=8)


In [ ]:
# get theoretical values along line in thetas phis

# zi_3, theta_gc_3, phi_gc_3 = get_values_line(my_sbs[inu], [thetas[inu, iTES, 0], phis[inu, iTES, 0]], [thetas[inu, idet, 1], phis[inu, iTES, 1]], npoints=npoints, delta_angle_deg=delta_angle_deg)
# zi_3, theta_gc_3, phi_gc_3 = get_values_line(sb, [thetas[inu, iTES, 1], phis[inu, iTES, 1]], [thetas[inu, iTES, 0], phis[inu, iTES, 0]], npoints=npoints, delta_angle_deg=delta_angle_deg)
sb_conv = hp.smoothing(sb, fwhm=np.radians(0.5))
# ind_0 = order_0
# ind_1 = order_1[0]
zi_3, theta_gc_3, phi_gc_3, angle_array_sb = pmf.get_values_line(sb_conv, [thetas[inu, iTES, ind_0], phis[inu, iTES, ind_0]], [thetas[inu, iTES, ind_1], phis[inu, iTES, ind_1]], npoints=npoints, ang_dist_deg=dist_order_0_1_deg, delta_angle_deg=delta_angle_deg)
print(np.shape(zi_3))

# fig, ax = plt.subplots()
# ax.plot(x_2, y_2)
# plt.show()

fig, ax = plt.subplots()
ax.plot(zi_3)
plt.show()

In [ ]:
# theta_gc2 = theta_gc + np.radians(4)
phi_gc2 = phi_gc + np.radians(4)
hp.gnomview(np.log10(sb/np.max(sb)), rot=[0,90], reso=20, min=-5, max=0,
             title='Theory {} {} GHz: TES #{}'.format(d['config'], d['filter_nu']/1e9,iTES))
variation = vals[inu, iTES,:]/np.max(vals[inu, iTES,:])
hp.projscatter(thetas[inu, iTES,:], phis[inu, iTES,:], c=variation, alpha=variation**(1/4), marker='x', cmap='Reds')
hp.projscatter(x_2, y_2, marker='o', c="r", lonlat=True)
hp.projscatter(theta_gc, phi_gc, marker='x', c="g")
# hp.projscatter(theta_gc2, phi_gc, marker='x', c="b")
hp.projscatter(theta_gc, phi_gc2, marker='x', c="b")
for i_peak in range(np.shape(thetas)[2]):
        hp.projtext(thetas[inu, iTES, i_peak], phis[inu, iTES, i_peak], 
                    '{0:5.0f}: {1:4.2f}'.format(i_peak, vals[i_nu, iTES,i_peak]/np.max(vals[i_nu, iTES,:])), c='w', fontsize=8)
plt.show()

In [ ]:
print(np.shape(great_circle_traj))

In [ ]:
%%script echo skipping

from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.plot3D(great_circle_traj[:, 0], great_circle_traj[:, 1], great_circle_traj[:, 2], 'blue')
ax.scatter3D(point_A[0], point_A[1], point_A[2], "red")
ax.scatter3D(point_B[0], point_B[1], point_B[2], "red")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
# ax.plot(zi_2/np.max(zi_2))
# ax.plot(zi/np.max(zi))
ax.plot(zi_3/np.max(zi_3), label="smoothed synthbeam at Moon fwhm")
ax.plot(zi_test/np.nanmax(zi_test), label="order 0 and 1 Moon peaks")
ax.plot(zi_test_diff/np.nanmax(zi_test_diff), label="same - neighbours")
# ax.plot(zi_3, label="smoothed synthbeam at Moon fwhm")
# ax.plot(zi_test_diff, label="same - neighbours")
plt.legend()
plt.show()

## Fit Moon spectrum

In [ ]:
min_point = 630
max_point = 880

print(np.degrees(angle_array_sb[95]))
print(np.degrees(angle_array_sb[min_point]))
print(np.degrees(angle_array_sb[max_point]))
print(cst.c*1e-9/(angle_array_sb[min_point]*delta_h))
print(cst.c*1e-9/(angle_array_sb[max_point]*delta_h))

In [ ]:
# start_point = np.array([theta_gc_test[0], phi_gc_test[0]])
# end_point = np.array([theta_gc_test[-1], phi_gc_test[-1]])
# theta_points_rad = np.linspace(-delta_angle_deg, np.linalg.norm(end_point - start_point) - delta_angle_deg, npoints)
# print(np.degrees(angle_array_test))
theta_points_rad = angle_array_sb #angle_array_test
# print(theta_points_rad)
points_nu_theo = np.zeros(n_nus)
points_nu_data = np.zeros(n_nus)
for i_nu, nu in enumerate(nus):
    # nu *= 1e9 # Hz
    bandwidth_lbda = cst.c / np.array([nu * (1 - rel_bw[i_nu]/2), nu * (1 + rel_bw[i_nu]/2)])
    mask_i = (theta_points_rad <= bandwidth_lbda[0]/delta_h) & (theta_points_rad >= bandwidth_lbda[1]/delta_h)
    print(cst.c * 1e-9 / bandwidth_lbda)
    print(np.arange(npoints)[mask_i])
    # data_nui = np.trapz(zi_test_diff[mask_i], x=theta_points_rad[mask_i])
    data_nui = np.mean(zi_test_diff[mask_i])
    points_nu_data[i_nu] = data_nui
    # theo_sb_nui = np.trapz(zi_3[mask_i], x=theta_points_rad[mask_i])
    theo_sb_nui = np.mean(zi_3[mask_i])
    points_nu_theo[i_nu] = theo_sb_nui

theta_nus_deg = np.degrees(cst.c / nus / delta_h)
theta_points_deg = np.degrees(theta_points_rad)
plt.figure()
plt.scatter(theta_nus_deg, points_nu_theo/np.max(zi_3), label="synthbeam")
plt.plot(theta_points_deg, zi_3/np.max(zi_3), label="sb")
plt.scatter(theta_nus_deg, points_nu_data/np.nanmax(zi_test_diff), label="data")
plt.plot(theta_points_deg, zi_test_diff/np.nanmax(zi_test_diff), label="sb")
plt.legend()
plt.show()

In [ ]:
theta_points_deg = np.degrees(theta_points_rad)
theta_nus_deg = np.degrees(cst.c / nus / delta_h)

fig, ax = plt.subplots(figsize=(12, 6))
# ax.plot(theta_points_deg, zi_3/np.max(zi_3), label="smoothed synthbeam at Moon fwhm")
# ax.plot(theta_points_deg, zi_test_diff/np.nanmax(zi_test_diff), label="same - neighbours")
ax.plot(theta_points_deg, zi_3, label="smoothed synthbeam at Moon fwhm")
# ax.scatter(theta_nus_deg, points_nu_theo/(cst.c/(rel_bw*nus*delta_h)), c="r")
ax.scatter(theta_nus_deg, points_nu_theo/(rel_bw*theta_points_rad), c="r")
plt.legend()
plt.show()

print(cst.c/(rel_bw*nus*delta_h))

In [ ]:
plt.figure()
plt.plot(theta_points_rad[1:] - theta_points_rad[:-1])
plt.ylim([0, 3e-4])
plt.show()

In [ ]:
peak_8_list = [False, False, False, False, False, False, False, False, False, False, False, False,
 False, False, False, False, False, False, False, False, False,  True, False, False,
 False,  True,  True, False, False, False, False, False,  True, False, False, False,
 False, False, False,  True, False, False, False, False, False,  True, False, False,
 False, False,  True,  True, False, False, False, False,  True, False, False, False,
  True,  True, False,  True, False, False,  True, False, False, False, False, False,
  True, False, False, False, False, False, False, False,  True, False,  True, False,
 False,  True,  True, False, False, False, False, False, False, False, False, False,
 False, False,  True, False, False, False, False, False, False, False,  True, False,
 False, False, False, False, False, False, False, False, False, False, False, False,
 False, False, False, False, False, False, False, False, False, False, False, False,
 False, False, False, False, False, False, False, False, False, False, False, False,
 False, False, False, False, False, False, False, False, False, False, False, False,
 False, False, False, False, False, False, False, False, False, False, False, False,
 False, False, False, False, False, False, False, False, False, False, False, False,
 False, False, False, False, False, False, False, False, False, False, False, False,
 False, False, False, False, False, False, False, False, False, False, False, False,
 False, False, False, False, False, False, False, False, False, False, False, False,
 False, False, False, False, False, False, False, False, False, False, False, False,
 False, False, False, False, False, False, False, False, False, False, False, False,
 False, False, False, False, False, False, False, False, False, False, False, False,
 False, False, False, False,]

In [ ]:
doplot = False

# npoints = 200
# delta_angle_deg = 1 # take a bit more at each end of line
extra_shift = 0 #np.radians(0.05) # radians # shift applied on real data positions to fit the theoretical positions (why?)

central_freq = 150.
numin = central_freq * (1 - d['filter_relative_bandwidth']/2.)
numax = central_freq * (1 + d['filter_relative_bandwidth']/2.)
n_nus = 5
nus = np.linspace(numin, numax, n_nus)
nside = nside
npix = 12 * d["nside"]**2
ndet = np.sum(peak_8_list)
all_values_sbs = np.zeros((npoints, ndet))
mean_val_sb = np.zeros((npoints))

# ind_0 = order_0 # index of order 0 in thetas, phis, vals arrays
# ind_1 = order_1[0] # index of one of the order 1 in thetas, phis, vals arrays

values_mean = np.zeros(npoints)
idet = -1
for i_TES_, isokTES in enumerate(peak_8_list):
    if not isokTES:
        continue
    idet += 1
    map_i = allmaps[i_TES_]
    order_0_pos = moon_fit[i_TES_][0]
    order_1_pos_end = order_0_pos + (np.array([7.35, -6.17])) # test of a general direction for all TES
    order_0_pos_sph = np.array([np.radians(90 - order_0_pos[1]), np.radians(order_0_pos[0])]) - extra_shift
    order_1_pos_end_sph = np.array([np.radians(90 - order_1_pos_end[1]), np.radians(order_1_pos_end[0])]) - extra_shift
    # zi_test, theta_gc_test_i, phi_gc_test_i = get_values_line(map_i, order_0_pos_sph, order_1_pos_end_sph, npoints=npoints, delta_angle_deg=delta_angle_deg)
    _, theta_gc_test_i, phi_gc_test_i, _ = get_values_line(map_i, order_0_pos_sph, order_1_pos_end_sph, npoints=npoints, delta_angle_deg=delta_angle_deg)
    zi_test, _, _, _ = get_values_large_line(map_i, theta_gc_test_i, phi_gc_test_i, width_angle_deg=0.1, nlines=3)
    nlines = 4
    theta_gc_test_diff = np.tile(theta_gc_test_i, (nlines, 1))
    phi_gc_test_diff_up = phi_gc_test_i + np.radians(np.linspace(2, 3, nlines)).reshape(nlines, 1)
    phi_gc_test_diff_down = phi_gc_test_i - np.radians(np.linspace(2, 3, nlines)).reshape(nlines, 1)
    zi_test2 = hp.get_interp_val(map_i, theta_gc_test_diff, phi_gc_test_diff_up)
    zi_test3 = hp.get_interp_val(map_i, theta_gc_test_diff, phi_gc_test_diff_down)
    zi_test2_mean = np.median(zi_test2, axis=0)
    zi_test3_mean = np.median(zi_test3, axis=0)
    concatenated_zi = np.concatenate([zi_test2, zi_test3])
    concatenated_zi[concatenated_zi**2 > 1e20] = np.NaN
    mean_concatenated_zi = np.mean(concatenated_zi, axis=0)
    zi_test_diff = zi_test - mean_concatenated_zi
    values_mean += zi_test

    if doplot:
        hp.gnomview(map_i, reso=8, rot=(0, 0, 0), return_projected_map=True, no_plot=False).data
        hp.projscatter(theta_gc_test_i, phi_gc_test_i, marker='.', c="g", lonlat=False)
        hp.projscatter(theta_gc_test_i, phi_gc_test_i + np.radians(2), marker='.', c="b", lonlat=False)
        hp.projscatter(theta_gc_test_i, phi_gc_test_i - np.radians(2), marker='.', c="b", lonlat=False)
        plt.show()

    sb_idet = np.zeros((npix))
    for inu in range(n_nus):
        d['filter_nu'] = nus[inu] * 1e9
        q_instrument_i = QubicInstrument(d)
        q_scene_i = QubicScene(d)
        sb_idet += q_instrument_i.get_synthbeam(q_scene_i, i_TES_)
    sb_smooth_i = hp.smoothing(sb_idet, fwhm=np.radians(0.5))
    # values_sb_i, _, _ = get_values_line(sb_smooth_i, [thetas[0, i_TES_, ind_0], phis[0, i_TES_, ind_0]], [thetas[0, i_TES_, ind_1], phis[0, i_TES_, ind_1]], npoints=npoints, delta_angle_deg=delta_angle_deg)
    _, theta_gc_i, phi_gc_i, _ = get_values_line(sb_smooth_i, [thetas[0, i_TES_, ind_0], phis[0, i_TES_, ind_0]], [thetas[0, i_TES_, ind_1], phis[0, i_TES_, ind_1]], npoints=npoints, delta_angle_deg=delta_angle_deg)
    values_sb_i, _, _, _ = get_values_large_line(sb_smooth_i, theta_gc_i, phi_gc_i, width_angle_deg=0.1, nlines=3)
    all_values_sbs[:, idet] = values_sb_i
    mean_val_sb += values_sb_i

    scaled_signal = zi_test_diff/np.nanmax(zi_test_diff)
    scaled_sb = plot_sb/np.max(plot_sb)
    mask_01 = scaled_sb < 0.1

    plot_abscisses = np.arange(npoints)

    fig, ax = plt.subplots(figsize=(12, 6))
    plot_sb = all_values_sbs[:, idet]/n_nus
    ax.set_title("TES {}".format(i_TES_ + 1))
    ax.plot(scaled_sb, label="smoothed synthbeam at Moon fwhm")
    ax.plot(plot_abscisses[mask_01], (scaled_sb)[mask_01], c="y", ls="--")
    ax.plot(zi_test/np.max(zi_test), label="order 0 and 1 Moon peaks")
    ax.plot(scaled_signal, label="same - neighbours")
    ax.plot(plot_abscisses[mask_01], scaled_signal[mask_01], c="y", ls="--", label="< 0.01 signal")
    ax.plot(mean_concatenated_zi/np.max(zi_test), label="neighbours")
    plt.legend()
    plt.show() 

values_mean /= ndet
mean_val_sb /= ndet
sum_mean_val_sb = np.sum(mean_val_sb, axis=0)
